In [82]:
import os 
import sys
import datetime
import csv 

import cirq
import qsimcirq
import openfermion
import numpy as np
import multiprocessing as mp
from functools import partial

Pi=3.1415

## anzats

## expectation

## optimization

In [83]:
def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("bata[{}]".format(p))
    print(headline)
    textlines.append(headline)

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            writer = csv.writer(f)
            for i, textline in enumerate(textlines):
                writer.writerow(textline)
                # f.write("{}\n".format(textline))

    return gamma, beta

In [84]:
class AnzatsAFMHeisenbergLattice():
    def __init__(self, rows, cols, gamma, beta):
        # Initialize circuit
        circuit = cirq.Circuit()
        qubits = [cirq.GridQubit(i, j) for i in range(rows) for j in range(cols)]
        
        # Symmetry cols=4, rows=2
        # q(0,0) q(0,1) q(0,2) q(0,3)
        # q(1,0) q(1,1) q(1,2) q(1,3)
       
        for i in range(rows):
            for j in range(cols):
                if (i * cols + j) % 2 == 0:  # Even qubits
                    circuit.append(cirq.H(qubits[i * cols + j]))
                    circuit.append(cirq.Y(qubits[i * cols + j]))
                else:  # Odd qubits
                    circuit.append(cirq.X(qubits[i * cols + j]))
                    
        for i in range(cols // 2):             
            #Correlation
            circuit.append(cirq.CNOT(qubits[2*i], qubits[2*i + 1]))
            circuit.append(cirq.CNOT(qubits[2*i], qubits[2*i + cols]))
            circuit.append(cirq.CNOT(qubits[2*i + cols], qubits[2*i + 1 + cols]))
            circuit.append(cirq.CNOT(qubits[2*i + 1], qubits[2*i + 1 + cols]))
                    
        def add_correlation_gates(circuit, qubit1, qubit2, gamma, beta):
            circuit.append(cirq.XX(qubit1, qubit2) ** (-gamma * 2 / Pi))
            circuit.append(cirq.YY(qubit1, qubit2) ** (-gamma * 2 / Pi))
            circuit.append(cirq.ZZ(qubit1, qubit2) ** (-gamma * 2 / Pi))
            circuit.append(cirq.XX(qubit1, qubit2) ** (-beta * 2 / Pi))
            circuit.append(cirq.YY(qubit1, qubit2) ** (-beta * 2 / Pi))
            circuit.append(cirq.ZZ(qubit1, qubit2) ** (-beta * 2 / Pi))
        
        for index in range(gamma.size):
            #print('Start')
            # Add gates between qubits q(1) and q(2)
            #print('1--------------------------------------------------------------------------')
            for i in range(1, cols, 2):  # Odd qubits
                for j in range(rows - 1):  # First row
                    current_index = j * cols + i                    
                    right_neighbor = j * cols + (i + 1) % cols
                    add_correlation_gates(circuit, qubits[current_index], qubits[right_neighbor], gamma[index], beta[index])
                    #print(f'Adding correlation between qubits {current_index} and {right_neighbor}...')

            # Add gates between qubits q(0) and q(1) + q(2) and q(3)
            #print('2--------------------------------------------------------------------------')
            for i in range(0, cols, 2):
                for j in range(rows - 1):
                    current_index = j * cols + i
                    right_neighbor = j * cols + (i + 1) % cols
                    add_correlation_gates(circuit, qubits[current_index], qubits[right_neighbor], gamma[index], beta[index])
                    #print(f'Adding correlation between qubits {current_index} and {right_neighbor}...')
                    
            # Add gates between qubits q(5) and q(6)
            #print('3--------------------------------------------------------------------------')
            for i in range(1, cols, 2):  # Odd qubits
                for j in range(rows - 1):  # Number of rows -1
                    current_index = j * cols + i + cols                   
                    right_neighbor = j * cols + (i + 1) % cols + cols
                    add_correlation_gates(circuit, qubits[current_index], qubits[right_neighbor], gamma[index], beta[index])
                    #print(f'Adding correlation between qubits {current_index} and {right_neighbor}...')
                    
            # Add gates between qubits q(4) and q(5) + q(6) and q(7)
            #print('4--------------------------------------------------------------------------')
            for i in range(0, cols, 2):
                for j in range(rows - 1):
                    current_index = j * cols + i + cols
                    right_neighbor = j * cols + (i + 1) % cols + cols
                    add_correlation_gates(circuit, qubits[current_index], qubits[right_neighbor], gamma[index], beta[index])
                    #print(f'Adding correlation between qubits {current_index} and {right_neighbor}...')
                    
            # Add gates between qubits q(0) and q(4) + q(1) and q(5) + q(2) and q(6) + q(3) and q(7)
            #print('5--------------------------------------------------------------------------')
            for i in range(cols):
                for j in range(0, rows, 2):
                    current_index = j * cols + i
                    down_neighbor = ((j + 1) % rows) * cols + i 
                    add_correlation_gates(circuit, qubits[current_index], qubits[down_neighbor], gamma[index], beta[index])
                    #print(f'Adding correlation between qubits {current_index} and {down_neighbor}...')
            #print('End')
        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta

    def circuit_to_latex_using_qcircuit(self):
        return cirq.contrib.circuit_to_latex_using_qcircuit(
            self.circuit, self.qubits
        )


In [85]:
class AFMHeisenbergLatticeArgs():
    def __init__(self, rows, cols, qsim_option):
        self.rows = rows
        self.cols  = cols
        self.qsim_option = qsim_option

def get_expectation_afm_heisenberg_lattice(function_args, gamma, beta):
    # periodic boundary
    anzats = AnzatsAFMHeisenbergLattice(function_args.rows, function_args.cols, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)
    vector = simulator.simulate(circuit).state_vector()

    rows = function_args.rows
    cols  = function_args.cols
    value = 0 + 0j
    #print('Start')
    for i in range(rows):
        for j in range(cols):
            current_index = i * cols + j

            right_neighbor = i * cols + (j + 1) % cols
            #print(f'Adding gates to qubits {current_index} and {right_neighbor} (right)...')
            circuitX = anzats.circuit.copy()
            circuitY = anzats.circuit.copy()
            circuitZ = anzats.circuit.copy()

            circuitX.append(cirq.X(qubits[current_index]))
            circuitX.append(cirq.X(qubits[right_neighbor]))
            vector2 = simulator.simulate(circuitX).state_vector()
            value += np.dot(vector2.conj(), vector)

            circuitY.append(cirq.Y(qubits[current_index]))
            circuitY.append(cirq.Y(qubits[right_neighbor]))
            vector2 = simulator.simulate(circuitY).state_vector()
            value += np.dot(vector2.conj(), vector)

            circuitZ.append(cirq.Z(qubits[current_index]))
            circuitZ.append(cirq.Z(qubits[right_neighbor]))
            vector2 = simulator.simulate(circuitZ).state_vector()
            value += np.dot(vector2.conj(), vector)


            down_neighbor = ((i + 1) % rows) * cols + j
            #print(f'Adding gates to qubits {current_index} and {down_neighbor} (down)...')
            circuitX = anzats.circuit.copy()
            circuitY = anzats.circuit.copy()
            circuitZ = anzats.circuit.copy()

            circuitX.append(cirq.X(qubits[current_index]))
            circuitX.append(cirq.X(qubits[down_neighbor]))
            vector2 = simulator.simulate(circuitX).state_vector()
            value += np.dot(vector2.conj(), vector)

            circuitY.append(cirq.Y(qubits[current_index]))
            circuitY.append(cirq.Y(qubits[down_neighbor]))
            vector2 = simulator.simulate(circuitY).state_vector()
            value += np.dot(vector2.conj(), vector)

            circuitZ.append(cirq.Z(qubits[current_index]))
            circuitZ.append(cirq.Z(qubits[down_neighbor]))
            vector2 = simulator.simulate(circuitZ).state_vector()
            value += np.dot(vector2.conj(), vector)
    #print('End')
    return np.real(value)


## case: rows 2, columns 4, optimize gamma and beta

In [86]:
rows = 2
cols = 4
p=8
gamma = np.array([0.6, 0.6, 0.6, 0.6])
beta  = np.array([0.6, 0.6, 0.6, 0.6])

qsim_option = {'t': int(cols/2), 'f':1}
function_args = AFMHeisenbergLatticeArgs(rows, cols, qsim_option) # def __init__(self, rows, cols, qsim_option):

anzats = AnzatsAFMHeisenbergLattice(rows, cols, gamma, beta) # def __init__(self, rows, cols, gamma, beta):
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

# norm = np.dot(vector.conjugate(), vector)
# print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg_lattice(function_args, gamma, beta)
# print("expectation on afm: {}".format(value))

initial_gamma  = np.array([0.6 for _ in range(p)])
initial_beta  = np.array([0.6 for _ in range(p)])
iteration = 10
alpha = 0.1
delta_gamma = 0.0001
delta_beta  = 0.0001
gamma, beta = optimize_by_gradient_descent(
    partial(get_expectation_afm_heisenberg_lattice, 
    function_args=function_args), 
    initial_gamma, 
    initial_beta, 
    alpha, 
    delta_gamma, 
    delta_beta, 
    iteration)

                       ┌──┐       ┌──┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐                                                                           ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐                                                                           ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌────────────────────────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌

KeyboardInterrupt: 